In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [40]:
query = """SELECT 
  ano,
  esfera,
  poderes,
  quantidade_vinculos
FROM (
  SELECT 
    ano,
    CASE 
      WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
      WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
      WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
      ELSE 'Outros' 
    END AS esfera,
    CASE 
    WHEN natureza_juridica IN ('1015','1023', '1031','1104','1139','1112', '1147','1236','1120', '1155','1180','1341', '1163','1252','1260','1279','1287','1295','1309','1317','1325','1333','1171','1180', '1244', '1341') THEN 'Executivo'
    WHEN natureza_juridica IN ('1040','1058', '1066') THEN 'Legislativo'
    WHEN natureza_juridica IN ('1074','1082') THEN 'Judiciário'
    ELSE 'Outros' 
  END AS poderes,

    COUNT(*) AS quantidade_vinculos
  FROM `basedosdados.br_me_rais.microdados_vinculos`  
  GROUP BY ano, esfera,poderes
) subquery
WHERE esfera != 'Outros' and poderes!= 'Outros'
ORDER BY esfera, ano, poderes
"""


In [3]:
vinculos = '''repositoriodedadosgpsp.perfil_remuneracao.RAIS_vinculos_publicos'''

In [4]:
df = pandas_gbq.read_gbq(vinculos, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,sexo,poderes,tipo_adm,tipo_adm_detalhado,grau_instrucao,esfera,faixa_etaria,sigla_uf,carga_horaria,tipo_vinculo,quantidade_vinculos
0,2018,Feminino,Outros,Outros,Outros,Até Ensino Superior Completo,Outros,Entre 18 e 29 anos,RR,40,CLT,3
1,2012,Feminino,Outros,Outros,Outros,Até Ensino Médio,Outros,Entre 50 e 65 anos,PB,8,CLT,13
2,2006,Feminino,Outros,Outros,Outros,Até Fundamental,Outros,Entre 50 e 65 anos,GO,44,CLT,51
3,2016,Feminino,Outros,Outros,Outros,Até Ensino Médio,Outros,Entre 30 e 49 anos,RS,42,CLT,1
4,2016,Masculino,Outros,Outros,Outros,Até Ensino Médio,Outros,Acima de 65 anos,RJ,8,CLT,21
...,...,...,...,...,...,...,...,...,...,...,...,...
1547657,2017,Feminino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Ensino Superior Completo,Municipal,Entre 50 e 65 anos,SP,40,Contratado/tempo determinado,13
1547658,2010,Masculino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Fundamental,Municipal,Entre 50 e 65 anos,MG,40,Contratado/tempo determinado,12
1547659,2012,Feminino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Ensino Médio,Municipal,Entre 30 e 49 anos,SC,44,Contratado/tempo determinado,13
1547660,2021,Masculino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Ensino Médio,Municipal,Entre 50 e 65 anos,SC,44,Contratado/tempo determinado,3


In [6]:
df = df.sort_values(ascending=True, by= 'ano')
df

,ano,sexo,poderes,tipo_adm,tipo_adm_detalhado,grau_instrucao,esfera,faixa_etaria,sigla_uf,carga_horaria,tipo_vinculo,quantidade_vinculos
783034,2003,Feminino,Outros,Empresa Pública,Empresa Pública,Até Fundamental,Outros,Entre 30 e 49 anos,PE,40,CLT,32
813489,2003,Feminino,Judiciário,Adm Direta,Adm Direta,Até Fundamental,Federal,Entre 50 e 65 anos,BA,40,Estatutário,17
179134,2003,Masculino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Municipal,Entre 30 e 49 anos,AC,44,Contratado/prazo determinado,2
1526387,2003,Feminino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Municipal,Entre 30 e 49 anos,RS,22,Estatutário não efetivo,86
1177052,2003,Masculino,Outros,Outros,Outros,Até Ensino Superior Completo,Outros,Entre 30 e 49 anos,RJ,18,CLT,2
...,...,...,...,...,...,...,...,...,...,...,...,...
302787,2022,Masculino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Fundamental,Municipal,Entre 30 e 49 anos,RJ,32,Estatutário,1
302792,2022,Feminino,Outros,Indireta,Outros,Até Fundamental,Municipal,Acima de 65 anos,CE,44,Estatutário,2
1434715,2022,Masculino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Estadual,Entre 30 e 49 anos,RS,19,Contratado/tempo determinado,39
1434742,2022,Feminino,Executivo,Adm Direta,Adm Direta,Até Ensino Médio,Estadual,Acima de 65 anos,RS,18,Contratado/tempo determinado,1


In [7]:
dir = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar'


In [10]:
df

,ano,sexo,poderes,tipo_adm,tipo_adm_detalhado,grau_instrucao,esfera,faixa_etaria,sigla_uf,carga_horaria,tipo_vinculo,quantidade_vinculos
783034,2003,Feminino,Outros,Empresa Pública,Empresa Pública,Até Fundamental,Outros,Entre 30 e 49 anos,PE,40,CLT,32
813489,2003,Feminino,Judiciário,Adm Direta,Adm Direta,Até Fundamental,Federal,Entre 50 e 65 anos,BA,40,Estatutário,17
179134,2003,Masculino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Municipal,Entre 30 e 49 anos,AC,44,Contratado/prazo determinado,2
1526387,2003,Feminino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Municipal,Entre 30 e 49 anos,RS,22,Estatutário não efetivo,86
1177052,2003,Masculino,Outros,Outros,Outros,Até Ensino Superior Completo,Outros,Entre 30 e 49 anos,RJ,18,CLT,2
...,...,...,...,...,...,...,...,...,...,...,...,...
302787,2022,Masculino,Executivo,Indireta,Fundação pública de direito público ou autarquia,Até Fundamental,Municipal,Entre 30 e 49 anos,RJ,32,Estatutário,1
302792,2022,Feminino,Outros,Indireta,Outros,Até Fundamental,Municipal,Acima de 65 anos,CE,44,Estatutário,2
1434715,2022,Masculino,Executivo,Adm Direta,Adm Direta,Até Ensino Superior Completo,Estadual,Entre 30 e 49 anos,RS,19,Contratado/tempo determinado,39
1434742,2022,Feminino,Executivo,Adm Direta,Adm Direta,Até Ensino Médio,Estadual,Acima de 65 anos,RS,18,Contratado/tempo determinado,1


In [13]:
df.to_csv('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar\\RAIS_vinculos_publicos2022.csv',sep=';',encoding='latin-1', index=False)

In [41]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,esfera,poderes,quantidade_vinculos
0,1995,Estadual,Executivo,2654538
1,1995,Estadual,Judiciário,190478
2,1995,Estadual,Legislativo,84801
3,1996,Estadual,Executivo,2612218
4,1996,Estadual,Judiciário,253545
...,...,...,...,...
219,2020,Municipal,Legislativo,178823
220,2021,Municipal,Executivo,7021454
221,2021,Municipal,Legislativo,181270
222,2022,Municipal,Executivo,7426522


In [39]:
df = df.sort_values(ascending=True, by= 'ano')
df

,ano,sexo,poderes,grau_instrucao,esfera,sigla_uf,flag_publico_privado,quantidade_vinculos
6256,1994,Feminino,Outros,Até Ensino Superior Completo,Outros,PR,Privado,83174
5485,1994,Feminino,Outros,Ignorado,Outros,SC,Privado,364
704,1994,Masculino,Outros,Até Ensino Superior Completo,Outros,SE,Privado,7532
1232,1994,Feminino,Outros,Até Ensino Superior Completo,Outros,MG,Privado,105611
3002,1994,Feminino,Outros,Até Ensino Médio,Outros,AL,Privado,34670
...,...,...,...,...,...,...,...,...
14806,2021,Masculino,Outros,Até Fundamental,Estadual,PR,Público,746
3927,2021,Feminino,Outros,Até Fundamental,Outros,BA,Privado,65293
14808,2021,Masculino,Outros,Até Fundamental,Outros,PR,Público,1045
3947,2021,Masculino,Outros,Até Mestrado,Outros,BA,Privado,2474


In [29]:
df.columns

Index(['ano', 'variavel', 'categoria', 'media_remuneracao', 'fator_correcao',
       'media_remuneracao_ajustada'],
      dtype='object')

In [30]:
df['ano'].unique()

<IntegerArray>
[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Length: 20, dtype: Int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ano                         496 non-null    Int64  
 1   variavel                    496 non-null    object 
 2   categoria                   496 non-null    object 
 3   media_remuneracao           496 non-null    float64
 4   fator_correcao              496 non-null    float64
 5   media_remuneracao_ajustada  496 non-null    float64
dtypes: Int64(1), float64(3), object(2)
memory usage: 23.9+ KB


In [32]:
dir = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar'

In [33]:
df

,ano,variavel,categoria,media_remuneracao,fator_correcao,media_remuneracao_ajustada
0,2003,genero,Masculino,1590.814451,3.038449,4833.609149
1,2003,genero,Feminino,1066.453234,3.038449,3240.364146
2,2004,genero,Feminino,1181.802084,2.823838,3337.217273
3,2004,genero,Masculino,1771.140450,2.823838,5001.413167
4,2005,genero,Feminino,1282.466421,2.671812,3426.508679
...,...,...,...,...,...,...
491,2022,grau_instrucao,Até Ensino Médio,3505.378382,1.046200,3667.326863
492,2022,grau_instrucao,Ignorado,0.000000,1.046200,0.000000
493,2022,grau_instrucao,Até Fundamental,2582.674100,1.046200,2701.993643
494,2022,grau_instrucao,Até Doutorado,12640.010085,1.046200,13223.978551


In [34]:
os.chdir(dir) 

In [20]:
dir = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar'
df.to_csv('RAIS_evolucao_esfera.csv', sep=';', encoding='latin-1', index=False)

-----------------------------------------------------------

Testes

In [14]:
df = pd.read_csv("SIAPE_ingresso_aposentadoria_media.csv", encoding="latin-1", sep=';', decimal=',')
df

,tipo_aposentadoria,decada_aposentadoria,categoria_ministerios,media_contribuicao,quantidade_vinculos
0,Outros,1980,"Economia, Finanças, Planejamento, Previdência ...",26.0,3
1,Outros,1980,Educação e Ciência e Tecnologia,19.7,6
2,Outros,1980,Saúde,19.2,4
3,Outros,1990,"Cultura, Turismo e Esportes",12.0,1
4,Outros,1990,"Economia, Finanças, Planejamento, Previdência ...",26.6,5
...,...,...,...,...,...
200,Aposentadoria por incapacidade,2020,"Justiça, Segurança e Órgãos de Controle e Defesa",23.0,152
201,Aposentadoria por incapacidade,2020,"Meio ambiente, Agricultura e Agropecuária",27.2,37
202,Aposentadoria por incapacidade,2020,Presidência da República,20.1,51
203,Aposentadoria por incapacidade,2020,Relações Exteriores,29.9,13


Automatizando troca de csv

In [45]:
##os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [46]:
## baixando os atuais com , e exportando com ";"

In [47]:
## listando todos os itens do diretório
##len(glob.glob('*.csv'))

In [48]:
## Serve apenas na primeira vez . 

##for i in list(range(0,len(glob.glob('*.csv')))):
    
   # df= pd.read_csv(glob.glob('*.csv')[i])
    #df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')